In [2]:
import sys
sys.path.append("../")
sys.path.append("../../")
from dataloader.svdd_dataloader import CollisionLoader_new
import numpy as np
from sklearn.svm import OneClassSVM
import numpy as np
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from load_data import load_data
from deepod.metrics import ts_metrics
from deepod.metrics import point_adjustment 
%matplotlib inline

In [4]:
train_imu,train_audio,train_spec,val_imu,val_audio,val_spec,val_imu_normal,val_audio_normal,val_spec_normal = load_data()

train_combine = np.concatenate([np.mean(train_audio,axis=-1),train_imu],axis=1)

test_audio_all = np.concatenate([val_audio_normal,val_audio],axis=0)
test_imu_all   = np.concatenate([val_imu_normal,val_imu],axis=0)
test_combine = np.concatenate([np.mean(test_audio_all,axis=-1),test_imu_all],axis=1)

labels_all = np.concatenate([np.zeros(val_audio_normal.shape[0]),np.ones(val_audio.shape[0])])

In [10]:
clf_audio = KernelPCA(kernel='rbf', gamma=0.01).fit_transform(np.mean(test_audio_all,axis=-1))
clf_imu = KernelPCA(kernel='rbf', gamma=0.01).fit_transform(test_imu_all )
clf_all = KernelPCA(kernel='rbf', gamma=0.01).fit_transform(test_combine)

In [11]:
scores_audio      = pairwise_distances(clf_audio, metric='euclidean')
scores_imu        = pairwise_distances(clf_imu, metric='euclidean')
scores_all        = pairwise_distances(clf_all, metric='euclidean')

eval_metrics_audio = ts_metrics(labels_all, -scores_audio)
eval_metrics_imu = ts_metrics(labels_all, -scores_imu)
result_audio = eval_metrics_audio
result_imu = eval_metrics_imu
eval_metrics_all = ts_metrics(labels_all, -scores_all)
result_all = eval_metrics_all

print(result_audio)
print(result_imu)
print(result_all)

ValueError: y should be a 1d array, got an array of shape (1978, 1978) instead.

In [54]:
kpca_audio = KernelPCA(kernel='rbf', gamma=0.01)  
data_kpca_audio = kpca_audio.fit_transform(np.mean(np.concatenate([val_audio_normal,val_audio],axis=0),axis=-1))

kpca_imu = KernelPCA(kernel='rbf', gamma=0.01)  
data_kpca_imu = kpca_imu.fit_transform(np.concatenate([val_imu_normal,val_imu],axis=0))

distances_audio = pairwise_distances(data_kpca_audio, metric='euclidean')
anomalies_audio = np.where(distances_audio.mean(axis=1) < np.mean(distances_audio))[0]

distances_imu = pairwise_distances(data_kpca_imu, metric='euclidean')
anomalies_imu = np.where(distances_imu.mean(axis=1) < np.mean(distances_imu))[0]

In [55]:
distances_audio = pairwise_distances(data_kpca_audio, metric='euclidean')
anomalies_audio = np.where(distances_audio.mean(axis=1) < np.mean(distances_audio))[0]

distances_imu = pairwise_distances(data_kpca_imu, metric='euclidean')
anomalies_imu = np.where(distances_imu.mean(axis=1) < np.mean(distances_imu))[0]

In [56]:
common_anomalies = np.intersect1d(anomalies_audio, anomalies_imu)

total_samples = val_imu_normal.shape[0] + val_imu.shape[0]
ground_truth_labels = np.concatenate([np.ones(val_imu_normal.shape[0]), np.zeros(val_imu.shape[0])])

predictions = np.zeros(total_samples)
predictions[common_anomalies] = 1

accuracy = accuracy_score(ground_truth_labels, predictions)
precision = precision_score(ground_truth_labels, predictions)
recall = recall_score(ground_truth_labels, predictions)
f1 = f1_score(ground_truth_labels, predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.6160458452722063
Precision: 0.6897810218978102
Recall: 0.5080645161290323
F1 Score: 0.5851393188854489
